In [ ]:
# リスト 2.1.20
# Yahoo!ショッピング Web API を用いてカテゴリ ID 一覧を取得するための準備コード
# ------------------------------------------------------------------------------------------------
# 目的:
#   - カテゴリ検索エンドポイント `/ShoppingWebService/V1/json/categorySearch` を叩き、
#     親カテゴリIDから子カテゴリ（Children）を列挙する土台を用意する。
#   - このスニペットでは「API 呼び出し関数」と「カテゴリ抽出ジェネレータ」を定義する。
#
# 背景/注意:
#   - appid は Yahoo! デベロッパーネットワークで取得したアプリケーションIDを使用する（ダミー 'xxxx' のままでは動かない）。
#   - レート制御: `r_get` 内で 1 秒スリープしているが、正式なレート上限はドキュメントに従うこと。
#   - 例外処理: `get_cats` の `except: pass` は全例外を握りつぶすため、実運用ではログ/リトライへ改善推奨（ここでは挙動を変えない）。
#   - ネットワーク健全性: タイムアウト、リトライ、ステータスコード検査、User-Agent 指定等は本例では省略（最小例）。
#   - 文字コード: API 応答は JSON（UTF-8）想定。`requests` の `.json()` は応答の `Content-Type` を基にデコードする。
#   - 返却構造: `ResultSet` → `'0'` → `Result` → `Categories` → `Children` に子カテゴリがぶら下がる想定。
#     サービスの仕様変更や親カテゴリが末端の場合は `KeyError` が起き得る（ここでは例外を無視してスキップ）。
#
# 進展:
#   - `all_categories_file` は次のリストで CSV に保存する前提のプレースホルダ。本スニペット中では未使用。

import requests
import json
import time
import csv

# エンドポイント（JSON 版 categorySearch）
url_cat = "https://shopping.yahooapis.jp/ShoppingWebService/V1/json/categorySearch"

# アプリケーションID（p.34 の方法で取得した値を設定）
appid = "xxxx"

# すべてのカテゴリを書き出す CSV のパス（このリストでは未使用、後続で利用する想定）
all_categories_file = "./all_categories.csv"


def r_get(url, dct):
    """
    API リクエスト発行の薄いラッパ（最小実装）。

    パラメータ:
        url : str
            送信先エンドポイント。
        dct : dict
            クエリパラメータ。ここでは { 'appid': appid, 'category_id': <ID> } を想定。

    仕様/挙動:
        - レート抑制のため呼び出し毎に 1 秒スリープ。
        - 返り値は requests.Response（ステータス検査や例外処理は行わない）。
          実運用では:
            * timeout=... の指定
            * ステータスコード検査（.raise_for_status()）
            * 429/5xx への指数バックオフ
            * 固定 User-Agent の付与
          などを追加すること。
    """
    time.sleep(1)  # 単純なレート制御（1 リクエスト/秒）
    return requests.get(url, params=dct)


def get_cats(cat_id):
    """
    親カテゴリ ID を受け取り、その直下の子カテゴリを (子ID, タイトル辞書) の形で逐次返すジェネレータ。

    パラメータ:
        cat_id : str または int
            親カテゴリを表す ID。ルート（'1' 等）から辿るケースを想定。

    返却:
        yield (child_id, title_dict)
            child_id : str
                子カテゴリの ID。
            title_dict : dict
                'short' / 'medium' / 'long' をキーに持つタイトル表記群。

    実装メモ:
        - 応答 JSON の想定パス:
          ResultSet -> '0' -> Result -> Categories -> Children -> <連番キー> -> { Id, Title:{Short,Medium,Long} }
        - 仕様変更・末端カテゴリ・レスポンス異常時は例外が発生し得るため、
          現在は裸の except で握りつぶし、何も yield せずに終わる。
          本番では例外の種類ごとに:
            * ネットワーク: リトライ
            * JSON 構造: ログ/スキップ
            * 権限/パラメータ: 早期中断
          等の分岐を行うこと。
    """
    try:
        # API 叩き（appid と親 ID をクエリに付与）
        result = r_get(url_cat, {"appid": appid, "category_id": cat_id})

        # JSON にデコードし、想定の階層から子カテゴリ辞書を取り出す
        cats = result.json()["ResultSet"]["0"]["Result"]["Categories"]["Children"]

        # Children は { '0': {...}, '1': {...}, ..., '_container': ... } のような構造を取り得る
        for i, cat in cats.items():
            if i != "_container":  # メタ要素（コンテナ情報）はスキップ
                # 各子カテゴリの ID と 3 種のタイトル（短・中・長）を取り出して返す
                yield cat["Id"], {
                    "short": cat["Title"]["Short"],
                    "medium": cat["Title"]["Medium"],
                    "long": cat["Title"]["Long"],
                }
    except:
        # 例外の握りつぶし（最小例）。本番ではログやリトライに置き換えること。
        # 例: except requests.exceptions.RequestException as e: ... など
        pass

In [ ]:
# リスト 2.1.21
# カテゴリ一覧CSVファイルの生成
# ------------------------------------------------------------------------------------------------
# 目的:
#   - 既に定義済みの `get_cats(cat_id)`（親カテゴリID→直下の子カテゴリを列挙）を用いて、
#     Yahoo!ショッピングのカテゴリを「レベル1→2→3」の3層で全走査し、CSV に逐次書き出す。
#
# 前提:
#   - 直前のリスト 2.1.20 までで以下が定義済みであること:
#       * `all_categories_file` : 出力CSVのパス
#       * `get_cats(cat_id)`    : 子カテゴリ (Id, Title[Short/Medium/Long]) を yield するジェネレータ
#       * `csv` モジュールの import
#
# 出力仕様（1行あたりの列順）:
#   [カテゴリコードlv1, カテゴリコードlv2, カテゴリコードlv3,
#    カテゴリ名lv1,   カテゴリ名lv2,   カテゴリ名lv3,   カテゴリ名lv3_long]
#
# 実装メモ（挙動を変えずに背景のみ説明）:
#   - まずヘッダ行を書き出し、その後 3 重ループで L1→L2→L3 を辿り、バッファ `output_buffer` に
#     行を貯めた上で、都度 CSV に追記する（この例では L3 内側で即フラッシュ）。
#   - `get_cats` は内部でネットワーク I/O を伴い、例外を握りつぶしている（最小例）。実運用では
#     エラー分類（ネットワーク/構造/権限）に応じたリトライ・ログ化が望ましい。
#   - 本スニペットではファイルのエンコーディング指定を省略（Python 既定）。Excel 互換を重視する場合は
#     `encoding='utf-8-sig'` 等を利用するとよい（ここでは挙動を変えない）。
#   - `KeyError` の捕捉は L2 取得時に Children が存在しない等の構造差分を想定している。
# ------------------------------------------------------------------------------------------------

# ヘッダ定義（列名）
output_buffer = [
    [
        "カテゴリコードlv1",
        "カテゴリコードlv2",
        "カテゴリコードlv3",
        "カテゴリ名lv1",
        "カテゴリ名lv2",
        "カテゴリ名lv3",
        "カテゴリ名lv3_long",
    ]
]

# 既存ファイルを上書きモードで開き、まずはヘッダだけを書き出す
with open(all_categories_file, "w") as f:
    writer = csv.writer(
        f, lineterminator="\n"
    )  # 改行は \n に統一（Windows でも \r\n にならない）
    writer.writerows(output_buffer)
    output_buffer = []  # バッファをクリア（以降はデータ行を貯めて使う）

# カテゴリレベル1（ルート=1 から直下の子を列挙）
for id1, title1 in get_cats(1):
    print("カテゴリレベル１ :", title1["short"])
    try:
        # カテゴリレベル2（L1 の子を列挙）
        for id2, title2 in get_cats(id1):

            # カテゴリレベル3（L2 の子を列挙）
            for id3, title3 in get_cats(id2):
                # 1 レコード分を作成し、バッファに追加
                wk = [
                    id1,
                    id2,
                    id3,
                    title1["short"],
                    title2["short"],
                    title3["short"],
                    title3["long"],
                ]
                output_buffer.append(wk)

                # バッファを即フラッシュして CSV に追記
                # ここでは L3 の各反復ごとに書き出している（メモリ節約/クラッシュ時の消失低減）。
                with open(all_categories_file, "a") as f:
                    writer = csv.writer(f, lineterminator="\n")
                    writer.writerows(output_buffer)
                    output_buffer = []  # フラッシュ後は空に戻す

    except KeyError:
        # 例: ある L2 に Children が存在しない（末端）場合などの構造差分をスキップ
        continue

In [ ]:
# リスト 2.1.22
# CSVファイルの内容確認

import pandas as pd
from IPython.display import display

df = pd.read_csv(all_categories_file)
display(df.head())